In [5]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

  Using cached mlflow-3.3.1-py3-none-any.whl.metadata (30 kB)
  Using cached boto3-1.40.14-py3-none-any.whl.metadata (6.7 kB)
  Using cached awscli-1.42.14-py3-none-any.whl.metadata (11 kB)
  Using cached optuna-4.5.0-py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
!pip install -Uq 'lightgbm>=4.0.0'

In [10]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 5- Machine Learning Algorithms with HP tuning")

<Experiment: artifact_location='s3://campusxalidvcbucket/425591744193892905', creation_time=1755800992412, experiment_id='425591744193892905', last_update_time=1755800992412, lifecycle_stage='active', name='Exp 5- Machine Learning Algorithms with HP tuning', tags={}>

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

In [13]:
df = pd.read_csv('reddit_preprocessing.csv')

In [14]:
#Step1: Remap the class labels from [-1, 0, 1] to [2,0,1]
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

#Step2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Fill NaN values in 'clean_comment' with empty strings
df['clean_comment'] = df['clean_comment'].fillna('')

ngram_range = (1,3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features=max_features)
X=vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X, y)

#Step3: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)


#Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
  with mlflow.start_run():
     # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


#Step5: Optuna Objective function for XGBoost
def objective_lightbgm(trial):
  n_estimators = trial.suggest_int('n_estimators', 50, 300)
  learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
  max_depth = trial.suggest_int('max_depth', 3, 10)

  model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth = max_depth, random_state=42)
  return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


#Step7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
  study = optuna.create_study(direction = 'maximize')
  study.optimize(objective_lightbgm, n_trials = 30)

  #Get the best parameters and log only the best model
  best_params = study.best_params
  best_model = LGBMClassifier(n_estimators = best_params['n_estimators'], learning_rate = best_params['learning_rate'], max_depth = best_params['max_depth'], random_state=42)

  #log the model
  log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

run_optuna_experiment()

[I 2025-08-21 19:08:45,618] A new study created in memory with name: no-name-e7b93684-be56-4926-a065-67a2d95a20cc


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.227077 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99059
[LightGBM] [Info] Number of data points in the train set: 37850, number of used features: 970
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-21 19:09:05,687] Trial 0 finished with value: 0.7808306034027264 and parameters: {'n_estimators': 265, 'learning_rate': 0.03082578272098951, 'max_depth': 10}. Best is trial 0 with value: 0.7808306034027264.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.236785 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99059
[LightGBM] [Info] Number of data points in the train set: 37850, number of used features: 970
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-21 19:09:09,964] Trial 1 finished with value: 0.5957941456197823 and parameters: {'n_estimators': 63, 'learning_rate': 0.0027517435572243713, 'max_depth': 6}. Best is trial 0 with value: 0.7808306034027264.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.214663 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99059
[LightGBM] [Info] Number of data points in the train set: 37850, number of used features: 970
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-21 19:14:29,710] Trial 29 finished with value: 0.7846348937968932 and parameters: {'n_estimators': 162, 'learning_rate': 0.057479221040734586, 'max_depth': 10}. Best is trial 10 with value: 0.7973158617774491.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.247667 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99059
[LightGBM] [Info] Number of data points in the train set: 37850, number of used features: 970
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/08/21 19:14:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 19:15:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/425591744193892905/runs/a1ded57215824e6080916bb56248cf5d
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/425591744193892905


In [ ]:
# Load the dataframe.
df = pd.read_csv('/content/reddit_preprocessing.csv')

# Display the first 5 rows.
display(df.head())